In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#for Fernanda's part
%%bash

pip install PyPDF2
pip install nltk
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 63.4 MB/s eta 0:00:00


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
#initializing the document store:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [5]:
#Fernanda's code for loading the pdf and preprocessing
import PyPDF2
import fitz
import nltk
from collections import Counter
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

from google.colab import files
uploaded = files.upload()

def extract_text(file_path):
    # Using PyMuPDF to extract text
    text = ""
    with fitz.open(file_path) as pdf_file:
        for page_num in range(len(pdf_file)):
            page = pdf_file.load_page(page_num)
            text += page.get_text()
    return text

def extract_metadata(file_path):
    # Using PyMuPDF for extracting metadata
    with fitz.open(file_path) as pdf_file:
        metadata = pdf_file.metadata
        number_of_pages = len(pdf_file)
        return metadata, number_of_pages

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Remove common words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Saving test2.pdf to test2.pdf


In [10]:
#part to save the the extracted/preprocessed text to an output.txt file which goes to the document store
file_path = "test2.pdf"
text = extract_text(file_path)
metadata, number_of_pages = extract_metadata(file_path)

text = preprocess_text(text)

#write the text to output.txt
with open("output.txt", "w", encoding='utf-8') as text_file:
  text_file.write(text)

print("Text extracted, preprocessed and saved to output.txt!")

Text extracted, preprocessed and saved to output.txt!


In [11]:
#writing the document(s) to the documentstore
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

# Specify the path to the single text file you want to index
path_to_file = "output.txt"  # Replace with the actual file path
indexing_pipeline = TextIndexingPipeline(document_store)
# Directly pass the file path to run_batch, as it accepts both lists and individual paths
indexing_pipeline.run_batch(file_paths=[path_to_file])  # Enclose in a list for clarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 8/8 [00:00<00:00, 1705.61 docs/s]


{'documents': [<Document: {'content': 'simon gilchrist charles himmelberg boston university nber columbia university investment fundamentals finance 1 introduction well recognized financial variables cash flow cash stocks robust quantitatively important explanatory variables investment reducedform equations estimated firmlevel data following seminal work fazzari hubbard petersen 1988 large body recent empirical work attributes findings capital market imperfections see extensive survey hubbard 1998 literature argues access external debt equity costly internally available funds provide cheaper source financing thus increasing desired level investment cost premiums exter nal finance generally explained appealing models asym metric information agency problems despite volume empirical work literature financing based interpretations explanatory power cash flow financial variables investment equations remain controversial1 even among economists agree firms face degree financial frictions rema

In [12]:
#initializing the retriever
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [13]:
from haystack.nodes import FARMReader
#if use_gpu=True GPU accerelation (in Edit: Notebook settings) must be enabled
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [14]:
#creating the retriever-reader pipeline
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [15]:
#asking a question:
prediction = pipe.run(query="What is this report about?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

Inferencing Samples: 100%|██████████| 2/2 [00:02<00:00,  1.28s/ Batches]


In [16]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'investment fundamentals finance 255', 'type': 'extractive', 'score': 0.4617688059806824, 'context': 'lts 34 estimate much line ballpark figures 0103 discussed investment fundamentals finance 255 6 conclusions paper argue combining careful measurement ', 'offsets_in_document': [{'start': 8625, 'end': 8660}], 'offsets_in_context': [{'start': 58, 'end': 93}], 'document_ids': ['842d2ab2a48c064c9586de47cbaab614'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'standard deviation underwrit ing spreads seasoned equity issues', 'type': 'extractive', 'score': 0.4196796417236328, 'context': 'ts 4 oofin calomiris himmelberg 1998 report standard deviation underwrit ing spreads seasoned equity issues 58 annual data measured premium average lo', 'offsets_in_document': [{'start': 2276, 'end': 2339}], 'offsets_in_context': [{'start': 44, 'end': 107}], 'document_ids': ['b64a7572b2d559f9eb4e014da80f60b3'], 'meta': {'_split_id': 0}}>,
             <Answer {'

In [17]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`

'Query: What is this report about?'
'Answers:'
[   {   'answer': 'investment fundamentals finance 255',
        'context': 'lts 34 estimate much line ballpark figures 0103 discussed '
                   'investment fundamentals finance 255 6 conclusions paper '
                   'argue combining careful measurement '},
    {   'answer': 'standard deviation underwrit ing spreads seasoned equity '
                  'issues',
        'context': 'ts 4 oofin calomiris himmelberg 1998 report standard '
                   'deviation underwrit ing spreads seasoned equity issues 58 '
                   'annual data measured premium average lo'},
    {   'answer': 'finished goods correlation',
        'context': 'erence production sales smaller subset firms compustat '
                   'report finished goods correlation productiontocapital '
                   'ratio salestocapital ratio exceeds 099 li'},
    {   'answer': 'small versus large manufacturing firms',
        'context': 'ose valu